# Test Run-Through of the Training Routine

This notebook does a run-through of the training routine, using the methodology from `../model_and_training_fns/training_routine.py`, supported by `../model_and_training_fns/training_fns.py`.

Additionally, it describes issues that arose regarding the RMSProp Optimizer and the Distribution Loss function. It also attempts to debug why the models aren't learning effectively by performing checks on the training functions.

In [1]:
import sys
import os
import numpy as np
import torch
import torch.nn as nn
import math

sys.path.insert(0, os.path.abspath('../model_and_training_fns/'))
from model_definition import CNN
from dataloader import GW_DataSet, make_dataloader
import training_fns as pyt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Initiate the Model

In [2]:
model = CNN(local_effects_kernel=3,
            local_effects_neurons=8,
            local_effects_bias=True,
            local_effects_depth=3,
            activation=nn.ReLU,
            dense_neurons=32,
            dense_bias=True,
            dense_depth=4)

## About RMSProp As an Optimizer

From [*A log-additive neural model for spatio-temporal prediction of groundwater levels*](https://doi.org/10.1016/j.spasta.2023.100740) by Pagendam et. al.:

> [...] we update the parameters of the model using the RMSProp method [...] The parameter $\beta$ is a hyper-parameter chosen by the user, and in this implementation we used the default value (in the Keras API) of 0.9. Model training was performed using an initial learning rate of $\gamma = 1e -6$, and all model parameters were initialised randomly according to the Glorot uniform distribution method (Glorot and Bengio, 2010).

Keras Documentation for RMSProp: [https://keras.io/api/optimizers/rmsprop/](https://keras.io/api/optimizers/rmsprop/)

An Explanation of the Keras RMSProp: [https://python.plainenglish.io/keras-optimizers-explained-rmsprop-93febeaba374](https://python.plainenglish.io/keras-optimizers-explained-rmsprop-93febeaba374)

PyTorch Documentation for RMSProp: [https://pytorch.org/docs/stable/generated/torch.optim.RMSprop.html](https://pytorch.org/docs/stable/generated/torch.optim.RMSprop.html) 

*In conclusion...*

Keras `rho` == Pytorch $\alpha$ == Pytorch `alpha` == Pagendam $\beta$

In [3]:
optimizer = torch.optim.RMSprop(model.parameters(),
                                lr=1e-6, 
                                alpha=0.9, 
                                eps=1e-08, 
                                weight_decay=0, 
                                momentum=0, 
                                centered=False, 
                                foreach=None, 
                                maximize=False, 
                                differentiable=False)

## Setting Up the DataSets

In [4]:
train_dataset = GW_DataSet(os.path.join('..', '..', 'NPYdata', 'train'))
val_dataset = GW_DataSet(os.path.join('..', '..', 'NPYdata', 'tune'))
test_dataset = GW_DataSet(os.path.join('..', '..', 'NPYdata', 'test'))

In [5]:
train_val_summary_dict = {
                        "train_loss": [],
                        "val_loss": [],
                        "epoch_time": []
                        }
train_samples_dict = {
                        "epoch": [],
                        "batch": [],
                        "truth": [],
                        "mu1_prediction": [],
                        "mu2_prediction": [],
                        "log_sigma1_prediction": [],
                        "log_sigma2_prediction": [],
                        "loss": []
                        }
val_samples_dict = {
                        "epoch": [],
                        "batch": [],
                        "truth": [],
                        "mu1_prediction": [],
                        "mu2_prediction": [],
                        "log_sigma1_prediction": [],
                        "log_sigma2_prediction": [],
                        "loss": []
                        }

In [6]:
batch_size = 5
train_batches = 10
val_batches = 5
train_dataloader = make_dataloader(train_dataset, batch_size, train_batches)
val_dataloader = make_dataloader(val_dataset, batch_size, val_batches)

## About the Distrobution Loss Function

### From the paper

From [*A log-additive neural model for spatio-temporal prediction of groundwater levels*](https://doi.org/10.1016/j.spasta.2023.100740) by Pagendam et. al.:

>Specifically, our observations of groundwater depth are modelled as having a log-normal density:
>$$ f(y_i | x_{1, i}, x_{2, i}, \theta_1, \theta_2) = (2*\pi)^{-\frac{1}{2}} * (\sigma(x_{1, i}, x_{2, i}) * y_i)^{-1} * \exp({ -1 * \frac{(log(y_i) - \mu(x_{1, i}, x_{2, i}))^2}{2 \sigma^2(x_{1, i}, x_{2, i})}}) $$
>where $\mu(x_{1, i}, x_{2, i}) = \mu_1(x_{1, i}) + \mu_2(x_{2, i})$ and $\sigma^2(x_{1, i}, x_{2, i}) = \sigma^2_1(x_{1, i}) + \sigma^2_2(x_{2, i})$. As a result of this simple summation of means and variances, we refer to and as log-additive components of our model: outputs of the two architectures (i.e. the means and variances on Gaussian distributions on the log-scale) that can be added together to give the first and second moments of the predictive distribution on the log-scale.
>
>[...]
>
>The loss function over the batch can be written as:
>$$ \ell(y_i | x_{1, i}, x_{2, i}, \theta_1, \theta_2) = - \Sigma_{i=1}^{batchsize} log( f(y_i | x_{1, i}, x_{2, i}, \theta_1, \theta_2) ) $$

### From discussions with Pagendam

Pagendam describes implementing the loss function in `R` as follows:

`ll <- -0.5*K$square((mu - log(y_true))/(sigma)) - K$log(sigma) - log(y_true) - 0.5*log(2*pi)`

$$\ell = -0.5 * (\frac{\mu - log(y_{true})}{\sigma})^2 - log(\sigma) - log(y_{true}) - 0.5*log(2*\pi) $$

### Skylar's Naive Implementation

The model outputs as predictions $\mu_1$, $\mu_2$, $log(\sigma_1)$,and $log(\sigma_1)$. The first step is to consolidate values to $\mu$ and $\sigma^2$.

$$ \sigma_1 = e^{log(\sigma_1)}$$

$$ \sigma_2 = e^{log(\sigma_2)}$$

$$ \mu = \mu_1 + \mu_2 $$

$$ \sigma^2 = \sigma_1^2 + \sigma_2^2 $$

Skylar then implemented the loss function in `PyTorch` as:

`f = ((2*math.pi)**(-1/2)) * ((torch.sqrt(sigma_sqr) * y)**(-1)) * torch.exp((-1) * ((torch.log(y) - mu)**2) * ((2 * sigma_sqr)**(-1)))`

`torch.log(f)`

Or equivalently,

$$ f = (2*\pi)^{-\frac{1}{2}} * (\sqrt{\sigma^2} * y_{true})^{-1} * \exp(-1 * (log(y_{true}) - \mu)^2 * (2 * \sigma^2)^{-1} ) $$

$$ \ell = log(f) $$

#### Skylar's Attempt at Log Arithmatic

To convince themselves that their naive implementation was indeed equivalent to Pagendam's implementation, they did this bought of log arithmatic.

$$ log(f) = log\left( \color{lime}{(2*\pi)^{-\frac{1}{2}}} \color{white}{*} \color{cyan}{(\sqrt{\sigma^2} * y_{true})^{-1}} \color{white}{*} \color{yellow}{e^{ -1 * \frac{(log(y) - \mu)^2}{2 \sigma^2}}} \color{white}\right)$$

$$ log(f) = \color{lime}{log\left( (2*\pi)^{-\frac{1}{2}} \right)} \color{white}{+} \color{cyan}{log\left((\sqrt{\sigma^2} * y_{true})^{-1} \right)} \color{white}{+} \color{yellow}{log\left(e^{ -1 * \frac{(log(y) - \mu)^2}{2 \sigma^2}} \right)}$$

$$ log(f) = \color{lime}{-\frac{1}{2}log\left( (2*\pi) \right)} \color{white}{+} \color{cyan}{(-1)*log\left((\sqrt{\sigma^2} * y_{true}) \right)} \color{white}{+} \color{yellow}{(-1) * \frac{(log(y) - \mu)^2}{2 \sigma^2}}$$

$$ log(f) = \color{lime}{-\frac{1}{2} * log\left( (2*\pi) \right)} \color{white}{+} \color{cyan}{(-1)*log\left(\sigma\right) + (-1)*log\left( y_{true} \right)} \color{white}{+} \color{yellow}{(-1) * \frac{(log(y_{true}) - \mu)^2}{2 * \sigma^2}}$$

$$ log(f) = \color{lime}{-0.5* log(2\pi)} \color{cyan}{- log(\sigma) - log( y_{true})} \color{yellow}{- \frac{(log(y_{true}) - \mu)^2}{2 \sigma^2}}$$

$$ log(f) = \color{lime}{-0.5* log(2\pi)} \color{cyan}{- log(\sigma) - log( y_{true})}  \color{yellow}{-(\frac{1}{2}) \frac{(-1)^2(\mu - log(y_{true}))^2}{\sigma^2}}$$

$$ log(f) = \color{lime}{-0.5* log(2\pi)} \color{cyan}{- log(\sigma) - log( y_{true})}  \color{yellow}{-0.5 *  \left(\frac{\mu - log(y_{true}}{\sigma}\right)^2}$$

which matches Pagendam's implementation:

$$\ell \ell = \color{yellow}{-0.5 * (\frac{\mu - log(y_{true})}{\sigma})^2} \color{cyan}{- log(\sigma) - log(y_{true})} \color{lime}{- 0.5*log(2*\pi) }$$

Note that the loss 

$$\ell = - \Sigma_{i=1}^{batchsize} log(f)$$

so we need to multiply by $-1$ before we pass the loss to the next step in the network.

### Loss Function in `PyTorch`

These functions are from `training_fns.py` and could be imported as `pyt.Distribution_Loss` and `pyt.Distribution_Loss_v2` respectively

`Distribution_Loss` is the naive implementation of the loss function described in Pagendam et. al.

`Distribution_Loss_v2` is the implementation that matches Pagendam's implementation

In [7]:
class Distribution_Loss(nn.Module):
	def __init__(self):
		super(Distribution_Loss, self).__init__()

	def forward(self, preds: tuple[float, float, float, float], truth: float):
		"""
		Custom loss for groundwater data
		Args:
			pred (torch.tensor([[float, float, float, float], ...])): 
				shape: (batchsize, 4)
				array of prediction values (mu1, log_sigma1, mu2, log_sigma2) for batch of samples
			truth (torch.tensor([[float], ...])): 
				shape: (batchsize, 1)
				array of GW_depth (ground truth) for batch of samples
		Returns:
			loss (torch.tensor([[float], ...])): 
				shape: (batchsize, 1)
				array of loss values for batch of samples
		"""
		batchsize = np.shape(truth.cpu().detach().numpy().flatten())[0]
		loss = torch.zeros(batchsize)
		for i in range(batchsize):
			mu1, log_sigma1, mu2, log_sigma2 = preds[i]
			y = truth[i]
			sigma1 = torch.exp(log_sigma1)
			sigma2 = torch.exp(log_sigma2)
			mu = mu1 + mu2
			sigma_sqr = (sigma1**2) + (sigma2**2)
			f = ((2*math.pi)**(-1/2)) * ((torch.sqrt(sigma_sqr) * y)**(-1)) * torch.exp((-1) * ((torch.log(y) - mu)**2) * ((2 * sigma_sqr)**(-1)))
			loss[i] = torch.log(f)

		loss *= -1	
		return loss

In [18]:
class Distribution_Loss_v2(nn.Module):
    def __init__(self):
        super(Distribution_Loss_v2, self).__init__()
    
    def forward(self, preds: tuple[float, float, float, float], truth: float):
        """
        Custom loss for groundwater data
            Args:
                pred (torch.tensor([[float, float, float, float], ...])): 
                shape: (batchsize, 4)
                        array of prediction values (mu1, log_sigma1, mu2, log_sigma2) for batch of samples
                truth (torch.tensor([[float], ...])): 
                    shape: (batchsize, 1)
                    array of GW_depth (ground truth) for batch of samples
            Returns:
            loss (torch.tensor([[float], ...])): 
                shape: (batchsize, 1)
                array of loss values for batch of samples
        """
        batchsize = np.shape(truth.cpu().detach().numpy().flatten())[0]
        loss = torch.zeros(batchsize)
        for i in range(batchsize):
            mu1, log_sigma1, mu2, log_sigma2 = preds[i]
            y = truth[i]
            sigma1 = torch.exp(log_sigma1)
            sigma2 = torch.exp(log_sigma2)
            mu = mu1 + mu2
            sigma = torch.sqrt((sigma1**2) + (sigma2**2))
            loss[i] = -0.5*math.log(2*math.pi) - torch.log(sigma) - torch.log(y) -0.5*(( (mu - torch.log(y)) / sigma )**2)
            loss[i] *= -1
        
        return loss

In [19]:
loss_fn = Distribution_Loss_v2()

## Check if Weights are Updating

In [ ]:
initialweights = model.sub1.precipitation_branch.conv_layers[0].weight
for traindata in train_dataloader:
    truth, pred, train_loss = pyt.train_datastep(traindata, 
                                                model,
                                                optimizer,
                                                loss_fn,
                                                device)
finalweights = model.sub1.precipitation_branch.conv_layers[0].weight
torch.all(initialweights == finalweights)

... the weights are not updating. I'm not sure why.

## Check the `Train Epoch` Function

In [ ]:
summary_dict, train_sample_dict, val_sample_dict = pyt.train_epoch(train_dataloader,
                                                                    val_dataloader, 
                                                                    model,
                                                                    optimizer,
                                                                    loss_fn,
                                                                    train_val_summary_dict,
                                                                    train_samples_dict,
                                                                    val_samples_dict,
                                                                    device)

In [ ]:
train_sample_dict

## Check Functions that Save Things

In [ ]:
batch_size = 5
train_batches = 10
val_batches = 5
train_dataloader = make_dataloader(train_dataset, batch_size, train_batches)
val_dataloader = make_dataloader(val_dataset, batch_size, val_batches)

In [ ]:
trainbatches = len(train_dataloader)
valbatches = len(val_dataloader)
trainbatch_ID = 0
valbatch_ID = 0

In [ ]:
def append_to_dict(dictt: dict, batch_ID: int, truth, pred, loss):
	""" Function to appending sample information to a dictionary
		Dictionary must be initialized with correct keys

		Args:
			dictt (dict): dictionary to append sample information to
			batch_ID (int): batch ID number for samples
			truth (): array of truth values for batch of samples
			pred (): array of prediction values for batch of samples
			loss (): array of loss values for batch of samples

		Returns:
			dictt (dict): dictionary with appended sample information
	"""
	batchsize = np.shape(truth.cpu().detach().numpy().flatten())[0]
	for i in range(batchsize):
		dictt["epoch"].append(0) # To be easily identified later
		dictt["batch"].append(batch_ID)
		dictt["truth"].append(truth.cpu().detach().numpy().flatten()[i])
		mu1, log_sigma1, mu2, log_sigma2 = pred[i]
		dictt["mu1_prediction"].append(mu1.cpu().detach().numpy().flatten()[0])
		dictt["mu2_prediction"].append(mu2.cpu().detach().numpy().flatten()[0])
		dictt["log_sigma1_prediction"].append(log_sigma1.cpu().detach().numpy().flatten()[0])
		dictt["log_sigma2_prediction"].append(log_sigma2.cpu().detach().numpy().flatten()[0])
		dictt["loss"].append(loss[i].cpu().detach().numpy().flatten()[0])

	return dictt

In [ ]:
for traindata in train_dataloader:
    print('batch:', trainbatch_ID)
    trainbatch_ID += 1
    model.train()
    GW_depth = traindata['GW_depth'].to(torch.float32).unsqueeze(-1).to(device)
    preds = model(device, traindata)
    print('preds', preds)
    print('truth', GW_depth)
    loss = loss_fn(preds, GW_depth)
    print('loss', loss)
    optimizer.zero_grad()
    loss.mean().backward()
    optimizer.step()
    train_sample_dict = append_to_dict(train_samples_dict, trainbatch_ID, GW_depth, preds, loss)

In [ ]:
print('batchsize:', batch_size)
print('Predictions shape:', preds.shape)

## Check that the batches are correctly split into samaples

In [ ]:
for traindata in train_dataloader:
    model.eval()
    model.to(device)

    ## Run as a batch
    GW_depth = traindata['GW_depth'].to(torch.float32).unsqueeze(-1).to(device)
    preds = model(device, traindata)

    ##Split up into samples
    keys = traindata.keys()
    sample1 = {k: [] for k in keys}
    sample2 = {k: [] for k in keys}
    sample3 = {k: [] for k in keys}
    sample4 = {k: [] for k in keys}
    sample5 = {k: [] for k in keys}
    for k in keys:
        sample1[k] = traindata[k][0].unsqueeze(0)
        sample2[k] = traindata[k][1].unsqueeze(0)
        sample3[k] = traindata[k][2].unsqueeze(0)
        sample4[k] = traindata[k][3].unsqueeze(0)
        sample5[k] = traindata[k][4].unsqueeze(0)

    ## Run on Each Sample
    preds1 = model(device, sample1)
    preds2 = model(device, sample2)
    preds3 = model(device, sample3)
    preds4 = model(device, sample4)
    preds5 = model(device, sample5)
    split_preds = torch.cat((preds1, preds2, preds3, preds4, preds5), dim=0)

In [ ]:
split_preds == preds

In [ ]:
preds